In [1]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu
pip install -U streamlit
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

#setting up text splitter
chunkiness = 800
splitter = RecursiveCharacterTextSplitter(chunk_size=chunkiness,
                                               chunk_overlap=150)
#vector_db.save_local("faiss_index")
loader = PyPDFLoader("https://drive.google.com/uc?export=download&id=1FINANq4fgZ4WCCfqWjKrLL_xVYdsab_l")
pages = loader.load_and_split(text_splitter= splitter)

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

vector_db = FAISS.from_documents(pages, embeddings)

vector_db.save_local("faiss_index")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
%%writefile rag_app.py

from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Setting up text splitter
chunkiness = 800
splitter = RecursiveCharacterTextSplitter(chunk_size=chunkiness, chunk_overlap=150)

hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
vector_db_path = "/content/faiss_index"

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Load Vector Database
vector_db = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)

# Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

memory = ConversationBufferMemory(memory_key='chat_history',
                                  return_messages=True,
                                  output_key='answer')  # Set output_key to 'answer'

# Personality templates dictionary
personality_templates = {
    "Nutritionist": """
    You are a professional and knowledgeable nutritionist chatbot. Provide accurate and helpful advice using a calm and supportive tone. Use technical terms related to nutrition and health.

    Previous conversation:
    {chat_history}

    Context to answer question:
    {context}

    New human question: {question}
    Response:""",

    "Sassy Teenager": """
    You are a sassy and rebellious teenager named Emma. Respond with sarcasm, humor, and a bit of attitude. Feel free to include ironic jokes and dramatic expressions. Context is not your forte, and you love giving out-of-context and reckless responses.

    Previous conversation:
    {chat_history}

    Context to answer question:
    {context}

    New human question: {question}
    Response:""",

    "Hippie": """
    You are a totally chill and stoned New Age Hippie chatbot. Provide advice with a relaxed, groovy tone. Crack jokes, talk about the universe, and sprinkle in some far-out metaphors. Make everything sound like an epic cosmic journey.

    Previous conversation:
    {chat_history}

    Context to answer question:
    {context}

    New human question: {question}
    Response:"""
}


# Initialize default LLM and prompt template
llm = HuggingFaceEndpoint(repo_id=hf_model)
prompt = PromptTemplate(template=personality_templates["Nutritionist"],
                        input_variables=["context", "question"])

if "llm" not in st.session_state:
    st.session_state.llm = HuggingFaceEndpoint(
        repo_id=hf_model,
        temperature=0.01,
        repetition_penalty=1.03,
        top_p=0.95
    )
    st.session_state.prompt = PromptTemplate(template=personality_templates["Nutritionist"],
                                             input_variables=["context", "question"])

    st.session_state.memory = memory
    st.session_state.chain = ConversationalRetrievalChain.from_llm(
        st.session_state.llm,
        retriever=retriever,
        memory=st.session_state.memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={"prompt": st.session_state.prompt}
    )

# Function to update LLM and prompt template
def update_llm_and_prompt():
    global llm, prompt, chain, memory
    temperature = st.session_state.my_temperature
    llm = HuggingFaceEndpoint(
        repo_id=hf_model,
        temperature=temperature,
        repetition_penalty=1.03,
        top_p=0.95
    )
    personality = st.session_state.my_selectbox
    personality_template = personality_templates.get(personality, personality_templates["Nutritionist"])

    st.session_state.prompt = PromptTemplate(template=personality_template, input_variables=["context", "question"])


    memory = ConversationBufferMemory(memory_key='chat_history',
                                      return_messages=True,
                                      output_key='answer')
    chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={"prompt": st.session_state.prompt}
    )
    st.session_state.memory = memory
    st.session_state.chain = chain

##### Streamlit #####
st.title('NutriSaaS Bot')

def form_callback():
    update_llm_and_prompt()
    # Reset chat history in session state
    st.session_state.messages = []

col1, col2 = st.columns([1, 3], gap='large')

with col2:
    # Initialise chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat messages from history on app rerun
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # React to user input
    if prompt := st.chat_input("Healing with whole foods!"):
        # Display user message in chat message container
        st.chat_message("user").markdown(prompt)
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})
        # Begin spinner before answering question so it's there for the duration
        with st.spinner("Dont trust occidental medicine..."):
            # send question to chain to get answer
            answer = st.session_state.chain(prompt)
            # extract answer from dictionary returned by chain
            response = answer["answer"]
            # Display chatbot response in chat message container
            with st.chat_message("assistant"):
                st.markdown(answer["answer"])
            # Add assistant response to chat history
            st.session_state.messages.append({"role": "assistant", "content": response})

with col1:
    st.header('Chatbot Tuning')
    st.slider('Creativity', min_value=0.01, max_value=2.0, value=0.01, key='my_temperature', on_change=form_callback)
    st.write(f'Temperature set to: {st.session_state.my_temperature}')
    st.selectbox('Personality', ['Nutritionist', 'Sassy Teenager', 'Hippie'], key='my_selectbox', on_change=form_callback)
    st.write(f'Personality: {st.session_state.my_selectbox}')


Overwriting rag_app.py


In [ ]:
!streamlit run rag_app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.123.212.227
npx: installed 22 in 2.595s
your url is: https://funny-mangos-design.loca.lt
